# 1. Prepare a lot of files

In [1]:
import os

In [2]:
path = 'AA/'
files = ["wiki_0"+str(i) for i in range(9)] + ["wiki_"+str(i) for i in range(10, 14)]

In [3]:
def prep():
    count = 0
    for file in files:
        with open(path+file) as fp: 
            while True:
                markers=0
                empty=True
                with open('done/page'+str(count)+'.txt', 'a') as the_file:
                    while True:
                        line = fp.readline() 

                        if not line:
                            break
                        if line[0]=='<':
                            markers+=1
                            if markers==2:
                                break
                            continue
                        if empty and not line.isspace() and 'http' not in line:
                            empty = False
                        the_file.write(line)
                if empty:
                    os.remove('done/page'+str(count)+'.txt')
                    count-=1
                count+=1
                if not line:
                    break
# prep()

Downloaded a lot of pages from wikipedia xml export.\
Extracted xml to plain text using wiki extractor https://github.com/attardi/wikiextractor .\
Seperated into many files the function prep()

# 2. Prepare Bag-Of-Words
###### In our case union of all words
We will use Lemmatizer

In [4]:
# !pip install num2words
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')

In [5]:
import re
import num2words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
path = 'done/'
files = ['page'+str(i)+'.txt' for i in range(1388)]
bagOfWords = dict()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [7]:
def prepare(text):
    text = text.lower()
    text = text.replace("_","")
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub('  +',' ',text)
    text = re.sub(r'½','.5',text) # who tf uses this character?
    text = re.sub(r'¼','.25',text) # who tf uses this character?
    
    text_list = text.split()
    text = ""
    
    for word in text_list:

        word = lemmatizer.lemmatize(word, pos="v")

#         if word.isnumeric():
#             word = num2words.num2words(float(word))
        
        if word in stop_words: continue

        text += word + " "

    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'[0-9]','',text)
    
    return text

prepare Function - prepares given text for out bag-of-words, reducing words to their basic dictionary form

In [8]:
count=0
bagOfWords = dict() # slows down really quick, so remember to clear dict
for file in files:
    with open(path+file) as fp:
        for fileline in fp:
            line = prepare(fileline)
            for word in line.split():
                if word not in bagOfWords:
                    bagOfWords[word.lower()]=count
                    count+=1
                    if count % 1e4 == 0:
                        print(count, word)

10000 princesa
20000 obviously
30000 costcutting
40000 sponge
50000 wannabe
60000 gizona


Due to rather slow execution, print to check if everything is in order

# 3. For each document prepare bag-of-words and frequency

In [9]:
count=0
wordBags = [dict() for file in files]
for ix, file in enumerate(files):
    with open(path+file) as fp:
        for fileline in fp:
#             line = re.sub("[^\w]", " ", fileline) 
            line = prepare(fileline)
            for word in line.split():
                if word.lower() not in bagOfWords:
                    continue
                count+=1
                if count % 3e5 == 0:
                    print(count, word)
                if word not in wordBags[ix]:
                    wordBags[ix][word.lower()]=1
                else:
                    wordBags[ix][word.lower()]+=1

300000 coefficients
600000 since
900000 products


Print to assure process not hang

# 4. Create sparse term-by-document matrix

In [10]:
from scipy import sparse

In [11]:
A = sparse.lil_matrix((len(bagOfWords), len(wordBags))) # rows and cols
for ix, bag in enumerate(wordBags):
    for word in bag:
        A[bagOfWords[word], ix] = bag[word]

A - Matrix with the number of bag-of-words rows and number of documents columns

# 5. Calulate Inverse Document Frequency and multiply each matrix element

We want no amplify the strength of rare words / topic related words appearing in specific documents

In [12]:
import math

In [13]:
IDF=0
N=len(files)
nw=1
counted = A.getnnz(axis=1)
AIDF = sparse.lil_matrix((len(bagOfWords), len(wordBags))) # rows and cols

In [14]:
count=0
for word in bagOfWords:
    nw=counted[bagOfWords[word]]
    IDF = math.log(N/nw)
#     A[bagOfWords[word]]*=IDF
    AIDF[bagOfWords[word], :]=A[bagOfWords[word]]*IDF
    if count%1e4==0:
        print(word)
    count+=1
#     print(nw, IDF)

japan
subterranean
argument
yasuo
singlecelled
gangsta
manthesingabs


Print to ensure process not hang\
Remember to clear A matrix

# 6. Query
Input words and number of closest documents to display.

We will calculate similarity based on:
q - inputed bag-of-words\
$$ cos\theta_j = \frac{q^Td_j}{||q||*||d_j||} = \frac{q^TAe_j}{||q||*||Ae_j||} $$

In [15]:
import numpy as np

In [16]:
query = "Japan, Japanese and manga are super interesting and the best!"
k = 10
# query = "Black people in the Japan"
# k = 20
wantidf = True

Input

In [17]:
querybag = np.array([0 for i in bagOfWords])
text = prepare(query).split()


for word in text:
    if word not in bagOfWords:
        text.remove(word)
    else:
        querybag[bagOfWords[word]]+=1
print(len(bagOfWords), len(wordBags))

66262 1388


Input preperation

In [18]:
if wantidf:
    colA = AIDF.tocsc()
else:
    colA = A.tocsc()

res = []
mod = np.linalg.norm(querybag)

for i in range(len(wordBags)): # number of columns/files 
    modj = sparse.linalg.norm(colA[:,i])
    
    res.append(((querybag.T@colA[:,i])/(mod*modj) if modj!=0 else 0, i))
        
    if i%1e2==0:
        print(i)

res.sort(key=lambda x: (-x[0], -x[1]))
# res.sort()
res = res[:k]
print(res)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
[(array([0.46862835]), 0), (array([0.42606222]), 355), (array([0.40809564]), 360), (array([0.39063166]), 357), (array([0.37712339]), 2), (array([0.23110739]), 8), (array([0.22160013]), 6), (array([0.18880402]), 352), (array([0.17346298]), 1065), (array([0.15318436]), 354)]


Calculated results

In [19]:
print(f"For query \"{query}\", with key words {text}, the best {k} results are:\n")
for ix, i in enumerate(res):
    print(f"{ix+1}: file page{i[1]}.txt titled:")
    with open("done/page"+str(i[1])+".txt", "r") as f:
        print(f.readline())

For query "Japan, Japanese and manga are super interesting and the best!", with key words ['japan', 'japanese', 'manga', 'super', 'interest', 'best'], the best 10 results are:

1: file page0.txt titled:
Japan

2: file page355.txt titled:
Manga

3: file page360.txt titled:
Manga outside Japan

4: file page357.txt titled:
History of manga

5: file page2.txt titled:
Outline of Japan

6: file page8.txt titled:
Japan and World Bank

7: file page6.txt titled:
Black people in Japan

8: file page352.txt titled:
Hokusai Manga

9: file page1065.txt titled:
Interest articulation

10: file page354.txt titled:
Japan Media Arts Festival



# 7. Normalize vectors $d_j$ and $q$

Then aforementioned equation is simplified to
$$ |q^TA| = [|cos\theta_1|,|cos\theta_2|,\dots,|cos\theta_n|] $$

In [20]:
query = "Japan, Japanese and manga are super interesting and the best!"
k = 10
wantidf = True

Input

In [21]:
if wantidf:
    colA = AIDF.tocsc()
else:
    colA = A.tocsc()
querybag = np.array([0 for i in bagOfWords])
text = prepare(query).split()


for word in text:
    if word not in bagOfWords:
        text.remove(word)
    else:
        querybag[bagOfWords[word]]+=1
print(len(bagOfWords), len(wordBags))

66262 1388


Input processing

In [22]:
querybag = querybag/np.linalg.norm(querybag) #remember to clean!

for i in range(len(wordBags)):
    norm = sparse.linalg.norm(colA[:,i])
    colA[:,i] = colA[:,i]/norm
    if i%1e2 == 0:
        print(i, norm)

res = list(querybag.T@colA)
for i in range(len(res)):
    res[i]=(res[i], i)
res.sort(key=lambda x: (-x[0], -x[1]))
# res.sort()
res = res[:k]
print(res)

0 834.4732328418058
100 276.64221464214324
200 119.45020182290006
300 61.74315362409992
400 68.05379464717565
500 105.94348595433729
600 39.84752023403267
700 471.6939809301491
800 30.434632017645008
900 28.24674182487699
1000 70.67316282378326
1100 122.6489685848564
1200 273.0646039960876
1300 202.8068188279269
[(0.4686283454035747, 0), (0.4260622202042567, 355), (0.4080956408580059, 360), (0.39063166044005054, 357), (0.3771233910350561, 2), (0.23110738843627396, 8), (0.22160012726256922, 6), (0.18880402291936035, 352), (0.17346297981692485, 1065), (0.15318436118771325, 354)]


In [23]:
print(f"For query {query}, with key words {text}, the best {k} results are:")
for ix, i in enumerate(res):
    print(f"{ix+1}: file page{i[1]}.txt titled:")
    with open("done/page"+str(i[1])+".txt", "r") as f:
        print(f.readline())

For query Japan, Japanese and manga are super interesting and the best!, with key words ['japan', 'japanese', 'manga', 'super', 'interest', 'best'], the best 10 results are:
1: file page0.txt titled:
Japan

2: file page355.txt titled:
Manga

3: file page360.txt titled:
Manga outside Japan

4: file page357.txt titled:
History of manga

5: file page2.txt titled:
Outline of Japan

6: file page8.txt titled:
Japan and World Bank

7: file page6.txt titled:
Black people in Japan

8: file page352.txt titled:
Hokusai Manga

9: file page1065.txt titled:
Interest articulation

10: file page354.txt titled:
Japan Media Arts Festival



# 8. Noise reduction using SVD and low rank approximation

$$ A\simeq A_k = U_kD_kV^T_k = \sum_{i=1}^{k}\sigma_iu_iv_i^T $$

And probability measure being

$$ cos\phi_j = \frac{q^TA_ke_j}{||q||*||A_ke_j||} $$

In [24]:
def lowRankApp(k, A=AIDF):
    U,S,V = sparse.linalg.svds(A, k)
    return sparse.csc_matrix(U @ np.diag(S) @ V)

In [25]:
lowRankApp(150, A.tocsc()) # rather slow

<66262x1388 sparse matrix of type '<class 'numpy.float64'>'
	with 91971656 stored elements in Compressed Sparse Column format>

In [26]:
def test(query, k=10, frm=1, to=-1, stp=50, A=AIDF, noiseCancel=True, printf=False):
    colA = A.tocsc()
#     colA=A
    querybag = np.array([0 for i in bagOfWords])
    text = prepare(query).split()


    for word in text:
        if word not in bagOfWords:
            text.remove(word)
        else:
            querybag[bagOfWords[word]]+=1
            
        ### normalization
    querybag = querybag/np.linalg.norm(querybag) #remember to clean!
    for i in range(len(wordBags)):
        norm = sparse.linalg.norm(colA[:,i])
        colA[:,i] = colA[:,i]/norm


    for x in range(frm, min(colA.shape) if to==-1 else to, stp):
        if noiseCancel:
            APP = lowRankApp(x, colA)
        else:
            APP=colA
        res = list(querybag.T@APP)
        for i in range(len(res)):
            res[i]=(res[i], i)
        res.sort(key=lambda x: (-x[0], -x[1]))
        # res.sort()
        res = res[:k]
        res = [i[1] for i in res]
        print(f"For k={x} Top {k} results are: {res}")
    
    if printf:
        print(f"For query {query}, with key words {text}, the best {k} results are:")
        for ix, i in enumerate(res):
            print(f"{ix+1}: file page{i}.txt titled:")
            with open("done/page"+str(i)+".txt", "r") as f:
                print(f.readline())

In [27]:
query = "Japan, Japanese and manga are super interesting and the best!"
k = 10

I suggest using custom test(query, from=, to=, stp=) - stp=step fr k, the line below takes some time :)

In [28]:
test(query, frm=150, to=170, stp=30)

For k=150 Top 10 results are: [355, 360, 357, 0, 2, 8, 352, 6, 354, 359]


# 9. Compare reuslts with and without noise reduction. Find optimal k and test IDF's impact on results

In [29]:
query = "Japan, Japanese and manga are super interesting and the best!"
k = 10 # top k results are displayed - different k than the one is svd!

First lets see the results without noise cancell.\
The "For k=xx " deosn't matter in this case

In [30]:
test(query, k=k, frm=10, to=20, stp=30, noiseCancel=False)

For k=10 Top 10 results are: [0, 355, 360, 357, 2, 8, 6, 352, 1065, 354]


Next lets test many k values

In [31]:
test(query, k=k, stp=50)

For k=1 Top 10 results are: [862, 1218, 1277, 916, 1216, 1321, 1184, 930, 934, 1187]
For k=51 Top 10 results are: [355, 360, 357, 352, 338, 0, 337, 359, 2, 353]
For k=101 Top 10 results are: [355, 360, 357, 0, 2, 8, 352, 354, 6, 359]
For k=151 Top 10 results are: [355, 360, 357, 0, 2, 8, 352, 6, 354, 359]
For k=201 Top 10 results are: [355, 360, 0, 357, 2, 8, 6, 352, 354, 359]
For k=251 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=301 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=351 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=401 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 353]
For k=451 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 353]
For k=501 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=551 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=601 Top 10 results are: [355, 0, 360, 357, 2, 8, 6, 352, 359, 354]
For k=651 Top 10 results are: [355, 

Subjectively the best results are obtained for k ~ 150

##### For cells below, compile the Final version cell, that is further below, first!!!

First lets test without IDF

In [34]:
colA = A.tocsc()
### normalization
for i in range(len(wordBags)):
    norm = sparse.linalg.norm(colA[:,i])
    colA[:,i] = colA[:,i]/norm
    
APP=lowRankApp(150, colA.tocsc())
find(query, k, APP, True)

For query Japan, Japanese and manga are super interesting and the best!, with key words ['japan', 'japanese', 'manga', 'super', 'interest', 'best'], the best 10 results are:
1: file page355.txt titled:
Manga

2: file page0.txt titled:
Japan

3: file page2.txt titled:
Outline of Japan

4: file page357.txt titled:
History of manga

5: file page360.txt titled:
Manga outside Japan

6: file page6.txt titled:
Black people in Japan

7: file page8.txt titled:
Japan and World Bank

8: file page359.txt titled:
Kono Manga ga Sugoi!

9: file page352.txt titled:
Hokusai Manga

10: file page338.txt titled:
Anime



Next lets see outcomes of applying the proper way

In [35]:
colA = AIDF.tocsc()
### normalization
for i in range(len(wordBags)):
    norm = sparse.linalg.norm(colA[:,i])
    colA[:,i] = colA[:,i]/norm
    
APP=lowRankApp(150, colA.tocsc())
find(query, k, APP, True)

For query Japan, Japanese and manga are super interesting and the best!, with key words ['japan', 'japanese', 'manga', 'super', 'interest', 'best'], the best 10 results are:
1: file page355.txt titled:
Manga

2: file page360.txt titled:
Manga outside Japan

3: file page357.txt titled:
History of manga

4: file page0.txt titled:
Japan

5: file page2.txt titled:
Outline of Japan

6: file page8.txt titled:
Japan and World Bank

7: file page352.txt titled:
Hokusai Manga

8: file page6.txt titled:
Black people in Japan

9: file page354.txt titled:
Japan Media Arts Festival

10: file page359.txt titled:
Kono Manga ga Sugoi!



Not using IDF has resulted in different and less accurate outcome.\
IDF has impact on search results, it devalues popular words, making other (rarer, unique) have greater influence on the result.
###### IDF has huge impact, when we try to reduce noise using low rank approximation

# Final version

Due to slow SVD calculations, apply query, number of top results, already low rank approximated matrix and whether you want to print the titles of found articles

In [33]:
def find(query, k=10, A=AIDF, verbose=False):
#     colA = A.tocsc()
#     colA=A
    querybag = np.array([0 for i in bagOfWords])
    text = prepare(query).split()


    for word in text:
        if word not in bagOfWords:
            text.remove(word)
        else:
            querybag[bagOfWords[word]]+=1
            
        ### normalization
    querybag = querybag/np.linalg.norm(querybag) #remember to clean!
    for i in range(len(wordBags)):
        norm = sparse.linalg.norm(colA[:,i])
        colA[:,i] = colA[:,i]/norm

    res = list(querybag.T@APP)
    for i in range(len(res)):
        res[i]=(res[i], i)
    res.sort(key=lambda x: (-x[0], -x[1]))
    # res.sort()
    res = res[:k]
    res = [i[1] for i in res]
    
    if verbose:
        print(f"For query {query}, with key words {text}, the best {k} results are:")
        for ix, i in enumerate(res):
            print(f"{ix+1}: file page{i}.txt titled:")
            with open("done/page"+str(i)+".txt", "r") as f:
                print(f.readline())
    else:
        print(f"For k={x} Top {k} results are: {res}")

In [36]:
colA = AIDF.tocsc()
### normalization
for i in range(len(wordBags)):
    norm = sparse.linalg.norm(colA[:,i])
    colA[:,i] = colA[:,i]/norm
    
APP=lowRankApp(150, colA.tocsc())

In [37]:
query = "Japan, Japanese and manga are super interesting and the best!"
k = 10
find(query, k=k, A=APP, verbose=True)

For query Japan, Japanese and manga are super interesting and the best!, with key words ['japan', 'japanese', 'manga', 'super', 'interest', 'best'], the best 10 results are:
1: file page355.txt titled:
Manga

2: file page360.txt titled:
Manga outside Japan

3: file page357.txt titled:
History of manga

4: file page0.txt titled:
Japan

5: file page2.txt titled:
Outline of Japan

6: file page8.txt titled:
Japan and World Bank

7: file page352.txt titled:
Hokusai Manga

8: file page6.txt titled:
Black people in Japan

9: file page354.txt titled:
Japan Media Arts Festival

10: file page359.txt titled:
Kono Manga ga Sugoi!



You can change query and k and run a cell above for different searches - it works rather fast